In [1]:
%pip install ../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=32018 sha256=0d3ed3aae23475f6e010421734cb2c980388a57e69df2171a11d9e836c35a175
  Stored in directory: /tmp/pip-ephem-wheel-cache-qxzjtvf6/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()

In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.models import model_factory 
from spuco.utils import Trainer
from torch.optim import SGD

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-5, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

trainer.train(1)

Epoch 0: 100%|██████████| 938/938 [00:03<00:00, 300.29batch/s, accuracy=84.375%, loss=1.44] 


In [6]:
from spuco.group_inference import Cluster, ClusterAlg
from spuco.utils import get_class_labels

logits = trainer.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=get_class_labels(trainset),
    cluster_alg=ClusterAlg.KMEDOIDS,
    max_clusters=5,
    device=device,
    verbose=True
)

Getting Trainset Outputs: 100%|██████████| 938/938 [00:00<00:00, 1361.16batch/s]


In [7]:
group_partition = cluster.infer_groups()

Clustering class-wise:   0%|          | 0/5 [00:00<?, ?it/s]

Submodular Maximization: Selecting subset...


Sorting samples by exemplar: 100%|██████████| 11263/11263 [00:00<00:00, 354110.35it/s]


For n_clusters = 2 The average silhouette_score is : 0.3482431
Submodular Maximization: Selecting subset...


Sorting samples by exemplar: 100%|██████████| 11263/11263 [00:00<00:00, 320461.05it/s]


...

For n_clusters = 3 The average silhouette_score is : 0.13168326
Submodular Maximization: Selecting subset...


Sorting samples by exemplar: 100%|██████████| 12183/12183 [00:00<00:00, 262055.27it/s]


For n_clusters = 4 The average silhouette_score is : 0.10918761
Submodular Maximization: Selecting subset...


Clustering class-wise: 100%|██████████| 5/5 [01:42<00:00, 20.59s/it]

For n_clusters = 5 The average silhouette_score is : 0.10515751


In [8]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 4337
(0, 1) 8275
(0, 2) 53
(1, 0) 1714
(1, 1) 10375
(2, 0) 519
(2, 1) 27
(2, 2) 10717
(3, 0) 2871
(3, 1) 9270
(3, 2) 42
(4, 0) 1744
(4, 1) 10015
(4, 2) 41


In [9]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.74636845745907
Group (0, 1) Test Accuracy: 100.0
Group (0, 2) Test Accuracy: 0.0
Group (1, 0) Test Accuracy: 98.30805134189032
Group (1, 1) Test Accuracy: 99.82650602409639
Group (2, 0) Test Accuracy: 9.633911368015415
Group (2, 1) Test Accuracy: 0.0
Group (2, 2) Test Accuracy: 27.73164131753289
Group (3, 0) Test Accuracy: 99.16405433646813
Group (3, 1) Test Accuracy: 99.85976267529665
Group (3, 2) Test Accuracy: 0.0
Group (4, 0) Test Accuracy: 99.19724770642202
Group (4, 1) Test Accuracy: 100.0
Group (4, 2) Test Accuracy: 0.0


{(0, 0): 99.74636845745907,
 (0, 1): 100.0,
 (0, 2): 0.0,
 (1, 0): 98.30805134189032,
 (1, 1): 99.82650602409639,
 (2, 0): 9.633911368015415,
 (2, 1): 0.0,
 (2, 2): 27.73164131753289,
 (3, 0): 99.16405433646813,
 (3, 1): 99.85976267529665,
 (3, 2): 0.0,
 (4, 0): 99.19724770642202,
 (4, 1): 100.0,
 (4, 2): 0.0}

In [10]:
from torch.optim import SGD
from spuco.invariant_train import GroupBalanceERM
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 0:   0%|          | 0/938 [00:00<?, ?batch/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x256 and 400x120)

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()